In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
20792,2024-12-04,Polônia 1. Liga,16:00,Poznań,LKS Lodz,1.32,2.97,170.5,1.82,1.84,-9.5,1.95,1.72,A1SuIR43,0.757576,0.336700,0.549451,0.543478,0.094276,129.794,27.451269,0.211499,2.4,1.341641,0.559017,148.20,1.586,0.390935,0.246491,18.0,194.236,31.110238,0.160167,1.2,1.643168,1.369306,199.23,2.360,0.184255,0.078074,-23.0,76,87,1.95,2.29,123.126,0.000,0.543928,0.007728,0.088629,0.40,0.080,4.000000,0.738202,0.8,0.061798,0.00,0.000,inf,0.000000,0.0,0.000000
20793,2024-12-04,República Tcheca Nbl,14:00,Opava,Usti n. Labem,1.66,2.04,166.5,1.87,1.79,-3.5,2.03,1.67,z3t6AOeT,0.602410,0.490196,0.534759,0.558659,0.092606,329.236,302.609060,0.919125,0.6,1.341641,2.236068,858.40,4.350,4.083926,0.938834,-104.0,248.144,267.868235,1.079487,1.8,1.643168,0.912871,162.64,3.252,3.735702,1.148740,0.0,74,107,11.60,1.52,128.986,351.940,0.145244,0.030912,0.137599,-2.91,-0.582,-1.134021,0.716725,0.6,-0.116725,-1.58,-0.316,-3.291139,0.383196,0.3,-0.083196
20794,2024-12-04,Polônia 1. Liga,16:00,Slask Wroclaw II,Lancut,2.95,1.33,162.5,1.83,1.83,4.5,2.01,1.68,jwXXlTzH,0.338983,0.751880,0.546448,0.546448,0.090863,242.688,138.775643,0.571827,0.0,0.000000,NaN,178.45,3.612,1.957044,0.541817,-72.0,150.692,35.122385,0.233074,1.2,1.643168,1.369306,147.92,1.792,0.409231,0.228366,-19.0,83,86,2.15,1.72,191.242,0.000,0.535286,0.000000,0.126474,-0.25,-0.050,-39.000000,0.529838,0.6,0.070162,0.00,0.000,inf,0.000000,0.0,0.000000
20795,2024-12-04,Portugal Lfb Feminina,17:15,Galitos F,Gdessa Barreiro F,2.90,1.37,132.5,1.83,1.83,6.5,1.83,1.83,A3LIkpGG,0.344828,0.729927,0.546448,0.546448,0.074755,233.748,123.723158,0.529301,1.2,1.643168,1.369306,92.70,4.124,2.674786,0.648590,-28.0,122.996,58.023155,0.471748,1.8,1.643168,0.912871,146.40,1.724,0.806244,0.467659,29.0,90,80,1.03,1.83,190.182,98.648,0.506732,0.000000,0.000000,-0.14,-0.028,-67.857143,0.492726,0.6,0.107274,-1.18,-0.236,-1.567797,0.662522,0.5,-0.162522
20796,2024-12-04,Portugal Lfb Feminina,17:15,Uniao Sportiva F,Natacao F,1.49,2.55,133.5,1.83,1.83,-18.5,1.93,1.74,2wIAi6o4,0.671141,0.392157,0.546448,0.546448,0.063298,91.484,46.630733,0.509715,2.4,1.341641,0.559017,51.45,1.560,0.860930,0.551878,5.0,230.970,96.487893,0.417751,0.6,1.341641,2.236068,124.44,4.402,2.881210,0.654523,-120.0,35,68,1.47,1.83,94.888,286.242,0.371056,0.000000,0.073215,0.26,0.052,9.423077,0.782781,0.7,-0.082781,-3.17,-0.634,-2.444795,0.288891,0.2,-0.088891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20964,2024-12-04,Eua Ncaa,22:00,Iowa State,Marquette,1.37,2.99,151.5,1.80,1.91,-6.5,1.83,1.83,h0KQGAek,0.729927,0.334448,0.555556,0.523560,0.064375,130.392,34.100253,0.261521,1.8,1.643168,0.912871,112.86,1.580,0.496639,0.314328,67.0,126.288,32.920063,0.260675,3.0,0.000000,0.000000,179.54,1.568,0.327674,0.208976,91.0,99,94,1.14,1.91,104.270,169.806,0.525465,0.041931,0.000000,-0.16,-0.032,-11.562500,0.767830,0.9,0.132170,-1.93,-0.386,-5.155440,0.577128,0.7,0.122872
20965,2024-12-04,Eua Ncaa,22:00,Memphis,Louisiana Tech,1.18,4.89,147.5,1.80,1.91,-10.5,1.91,1.80,OU3ErbHq,0.847458,0.204499,0.555556,0.523560,0.051957,235.320,136.769342,0.581206,2

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
20797,17:30,Portugal Lfb Feminina,Sanjoanense F,Imortal F,1.92,Back Home
20859,21:30,Eua Nba,Philadelphia 76ers,Orlando Magic,2.36,Back Home
20863,22:30,Colômbia Lbp,Cimarrones del Choco,Motilones del Norte,1.54,Back Home
20871,21:00,Eua Ncaa,Lehigh,Monmouth (N.J.),1.95,Back Home
20879,22:00,Eua Ncaa,Tulsa,Oklahoma State,2.74,Back Home
20881,08:35,China Cba,Tianjin,Shandong,2.63,Back Home
20884,07:00,Coreia Do Sul Kbl,Wonju DB,Anyang JungKwanJang,1.57,Back Home
20917,21:30,Argentina Liga A,San Martin,Gimnasia,1.76,Back Home
20961,21:00,Eua Ncaa,Wofford,Gardner Webb,1.55,Back Home
